In [333]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime as dt
from sklearn.linear_model import LogisticRegression as lr

folder='/Users/gavin/Documents/Metis/Coursework/Project_3/'

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [113]:
df=pd.read_csv(folder+'data_sources/spreadspoke_scores.csv')

In [114]:
df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,09/02/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71,NaN
1,09/03/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70,NaN
2,09/04/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82,NaN
3,09/09/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78,NaN
4,09/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62,NaN


# Imported first database
Isolating to 2015-2019 & cleaning team names

In [115]:
df=df[df['schedule_season']>2014]

In [116]:
df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
11343,09/10/2015,2015,1,False,New England Patriots,28,21,Pittsburgh Steelers,NE,-7.0,51,Gillette Stadium,False,64.0,9.0,NaN,Rain
11344,09/13/2015,2015,1,False,Arizona Cardinals,31,19,New Orleans Saints,ARI,-2.5,48.5,University of Phoenix Stadium,False,72.0,0.0,NaN,DOME
11345,09/13/2015,2015,1,False,Buffalo Bills,27,14,Indianapolis Colts,IND,-1.0,44.5,Ralph Wilson Stadium,False,53.0,7.0,NaN,NaN
11346,09/13/2015,2015,1,False,Chicago Bears,23,31,Green Bay Packers,GB,-6.5,48.5,Soldier Field,False,68.0,4.0,NaN,NaN
11347,09/13/2015,2015,1,False,Dallas Cowboys,27,26,New York Giants,DAL,-7.0,52.5,Cowboys Stadium,False,72.0,0.0,NaN,DOME


In [117]:
df.team_home.unique()

array(['New England Patriots', 'Arizona Cardinals', 'Buffalo Bills',
       'Chicago Bears', 'Dallas Cowboys', 'Denver Broncos',
       'Houston Texans', 'Jacksonville Jaguars', 'New York Jets',
       'Oakland Raiders', 'San Diego Chargers', 'St. Louis Rams',
       'Tampa Bay Buccaneers', 'Washington Redskins', 'Atlanta Falcons',
       'San Francisco 49ers', 'Kansas City Chiefs', 'Carolina Panthers',
       'Cincinnati Bengals', 'Cleveland Browns', 'Green Bay Packers',
       'Minnesota Vikings', 'New Orleans Saints', 'New York Giants',
       'Philadelphia Eagles', 'Pittsburgh Steelers', 'Indianapolis Colts',
       'Baltimore Ravens', 'Detroit Lions', 'Miami Dolphins',
       'Seattle Seahawks', 'Tennessee Titans', 'Los Angeles Rams',
       'Los Angeles Chargers'], dtype=object)

In [118]:
abrev=df.team_favorite_id.unique()

# Creating one line per team per game, showing spread for that particular team

In [119]:
teams_home=[]
teams_away=[]
for item in df.team_home.unique():
    team_home=item.split(' ')[-1]
    teams_home.append(team_home)

for item in df.team_away.unique():
    team_away=item.split(' ')[-1]
    teams_away.append(team_away)

In [120]:
team_dict_home=dict(zip(df.team_home.unique(), teams_home))

In [121]:
team_dict_away = dict(zip(df.team_away.unique(),teams_away))

In [122]:
abrev_dict = dict(zip(df.team_favorite_id.unique(), ['Patriots', 'Cardinals', 'Colts', 'Packers', 'Cowboys', 'Broncos', 'Chiefs', 'Panthers', 'Jets', 'Raiders', 'Chargers', 'Seahawks', 'Buccaneers', 'Dolphins', 'Eagles', 'Vikings', 'Bills', 'Bengals', 'Titans', 'Saints', 'Giants', 'Ravens', 'Steelers', 'Rams', 'Browns', 'PICK', 'Texans', 'Falcons', 'Lions', 'Jaguars', 'Redskins', 'Bears', '49ers']))

In [123]:
df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
11343,09/10/2015,2015,1,False,New England Patriots,28,21,Pittsburgh Steelers,NE,-7.0,51,Gillette Stadium,False,64.0,9.0,NaN,Rain
11344,09/13/2015,2015,1,False,Arizona Cardinals,31,19,New Orleans Saints,ARI,-2.5,48.5,University of Phoenix Stadium,False,72.0,0.0,NaN,DOME
11345,09/13/2015,2015,1,False,Buffalo Bills,27,14,Indianapolis Colts,IND,-1.0,44.5,Ralph Wilson Stadium,False,53.0,7.0,NaN,NaN
11346,09/13/2015,2015,1,False,Chicago Bears,23,31,Green Bay Packers,GB,-6.5,48.5,Soldier Field,False,68.0,4.0,NaN,NaN
11347,09/13/2015,2015,1,False,Dallas Cowboys,27,26,New York Giants,DAL,-7.0,52.5,Cowboys Stadium,False,72.0,0.0,NaN,DOME


In [124]:
df1=df.copy()

In [125]:
df1['team_home_fix'] = df1['team_home'].map(team_dict_home)
df1['team_away_fix']=df1['team_away'].map(team_dict_away)
df1['team_favorite_id_fix']=df1['team_favorite_id'].map(abrev_dict)

In [126]:
df1.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_home_fix,team_away_fix,team_favorite_id_fix
11343,09/10/2015,2015,1,False,New England Patriots,28,21,Pittsburgh Steelers,NE,-7.0,51,Gillette Stadium,False,64.0,9.0,NaN,Rain,Patriots,Steelers,Patriots
11344,09/13/2015,2015,1,False,Arizona Cardinals,31,19,New Orleans Saints,ARI,-2.5,48.5,University of Phoenix Stadium,False,72.0,0.0,NaN,DOME,Cardinals,Saints,Cardinals
11345,09/13/2015,2015,1,False,Buffalo Bills,27,14,Indianapolis Colts,IND,-1.0,44.5,Ralph Wilson Stadium,False,53.0,7.0,NaN,NaN,Bills,Colts,Colts
11346,09/13/2015,2015,1,False,Chicago Bears,23,31,Green Bay Packers,GB,-6.5,48.5,Soldier Field,False,68.0,4.0,NaN,NaN,Bears,Packers,Packers
11347,09/13/2015,2015,1,False,Dallas Cowboys,27,26,New York Giants,DAL,-7.0,52.5,Cowboys Stadium,False,72.0,0.0,NaN,DOME,Cowboys,Giants,Cowboys


In [127]:
df_home = df1.copy()
df_away = df1.copy()

In [128]:
df_home['target']=df_home['team_home_fix']
df_away['target']=df_away['team_away_fix']

In [129]:
df_home.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_home_fix,team_away_fix,team_favorite_id_fix,target
11343,09/10/2015,2015,1,False,New England Patriots,28,21,Pittsburgh Steelers,NE,-7.0,...,Gillette Stadium,False,64.0,9.0,NaN,Rain,Patriots,Steelers,Patriots,Patriots
11344,09/13/2015,2015,1,False,Arizona Cardinals,31,19,New Orleans Saints,ARI,-2.5,...,University of Phoenix Stadium,False,72.0,0.0,NaN,DOME,Cardinals,Saints,Cardinals,Cardinals
11345,09/13/2015,2015,1,False,Buffalo Bills,27,14,Indianapolis Colts,IND,-1.0,...,Ralph Wilson Stadium,False,53.0,7.0,NaN,NaN,Bills,Colts,Colts,Bills
11346,09/13/2015,2015,1,False,Chicago Bears,23,31,Green Bay Packers,GB,-6.5,...,Soldier Field,False,68.0,4.0,NaN,NaN,Bears,Packers,Packers,Bears
11347,09/13/2015,2015,1,False,Dallas Cowboys,27,26,New York Giants,DAL,-7.0,...,Cowboys Stadium,False,72.0,0.0,NaN,DOME,Cowboys,Giants,Cowboys,Cowboys


In [130]:
df_home['target_spread']=0
df_away['target_spread']=0

In [131]:
df_home.loc[df_home['team_favorite_id_fix']==df_home['target'], 'target_spread']=df_home['spread_favorite']
df_home.loc[df_home['team_favorite_id_fix']!=df_home['target'], 'target_spread']=df_home['spread_favorite']*(-1)
df_away.loc[df_away['team_favorite_id_fix']==df_away['target'], 'target_spread']=df_away['spread_favorite']
df_away.loc[df_away['team_favorite_id_fix']!=df_away['target'], 'target_spread']=df_away['spread_favorite']*(-1)
df_home['home_away'] = 'H'
df_away['home_away']= 'A'


In [132]:
df_home.head(30)

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_home_fix,team_away_fix,team_favorite_id_fix,target,target_spread,home_away
11343,09/10/2015,2015,1,False,New England Patriots,28,21,Pittsburgh Steelers,NE,-7.0,...,64.0,9.0,NaN,Rain,Patriots,Steelers,Patriots,Patriots,-7.0,H
11344,09/13/2015,2015,1,False,Arizona Cardinals,31,19,New Orleans Saints,ARI,-2.5,...,72.0,0.0,NaN,DOME,Cardinals,Saints,Cardinals,Cardinals,-2.5,H
11345,09/13/2015,2015,1,False,Buffalo Bills,27,14,Indianapolis Colts,IND,-1.0,...,53.0,7.0,NaN,NaN,Bills,Colts,Colts,Bills,1.0,H
11346,09/13/2015,2015,1,False,Chicago Bears,23,31,Green Bay Packers,GB,-6.5,...,68.0,4.0,NaN,NaN,Bears,Packers,Packers,Bears,6.5,H
11347,09/13/2015,2015,1,False,Dallas Cowboys,27,26,New York Giants,DAL,-7.0,...,72.0,0.0,NaN,DOME,Cowboys,Giants,Cowboys,Cowboys,-7.0,H
11348,09/13/2015,2015,1,False,Denver Broncos,19,13,Baltimore Ravens,DEN,-4.5,...,86.0,5.0,NaN,NaN,Broncos,Ravens,Broncos,Broncos,-4.5,H
11349,09/13/2015,2015,1,False,Houston Texans,20,27,Kansas City Chiefs,KC,-1.0,...,83.0,8.0,NaN,DOME (Open Roof),Texans,Chiefs,Chiefs,Texans,1.0,H
11350,09/13/2015,2015,1,False,Jacksonville Jaguars,9,20,Carolina Panthers,CAR,-3.0,...,81.0,1.0,NaN,NaN,Jaguars,Panthers,Panthers,Jaguars,3.0,H
11351,09/13/2015,2015,1,False,New York Jets,31,10,Cleveland Browns,NYJ,-3.5,...,76.0,7.0,NaN,NaN,Jets,Browns,Jets,Jets,-3.5,H
11352,09/13/2015,2015,1,False,Oakland Raiders,13,33,Cincinnati Bengals,OAK,-3.0,...,75.0,9.0,NaN,NaN,Raiders,Bengals,Raiders,Raiders,-3.0,H


In [135]:
df_home['target_adj_score']=df_home['score_home']+df_home['target_spread']
df_away['target_adj_score']=df_away['score_away']+df_away['target_spread']
df_home['opposition_score']=df_home['score_away']
df_away['opposition_score']=df_away['score_home']

In [136]:
df_home.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,weather_humidity,weather_detail,team_home_fix,team_away_fix,team_favorite_id_fix,target,target_spread,home_away,target_adj_score,opposition_score
11343,09/10/2015,2015,1,False,New England Patriots,28,21,Pittsburgh Steelers,NE,-7.0,...,NaN,Rain,Patriots,Steelers,Patriots,Patriots,-7.0,H,21.0,21
11344,09/13/2015,2015,1,False,Arizona Cardinals,31,19,New Orleans Saints,ARI,-2.5,...,NaN,DOME,Cardinals,Saints,Cardinals,Cardinals,-2.5,H,28.5,19
11345,09/13/2015,2015,1,False,Buffalo Bills,27,14,Indianapolis Colts,IND,-1.0,...,NaN,NaN,Bills,Colts,Colts,Bills,1.0,H,28.0,14
11346,09/13/2015,2015,1,False,Chicago Bears,23,31,Green Bay Packers,GB,-6.5,...,NaN,NaN,Bears,Packers,Packers,Bears,6.5,H,29.5,31
11347,09/13/2015,2015,1,False,Dallas Cowboys,27,26,New York Giants,DAL,-7.0,...,NaN,DOME,Cowboys,Giants,Cowboys,Cowboys,-7.0,H,20.0,26


In [137]:
df2=pd.concat([df_home, df_away], ignore_index=True)

In [138]:
df2.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,weather_humidity,weather_detail,team_home_fix,team_away_fix,team_favorite_id_fix,target,target_spread,home_away,target_adj_score,opposition_score
0,09/10/2015,2015,1,False,New England Patriots,28,21,Pittsburgh Steelers,NE,-7.0,...,NaN,Rain,Patriots,Steelers,Patriots,Patriots,-7.0,H,21.0,21
1,09/13/2015,2015,1,False,Arizona Cardinals,31,19,New Orleans Saints,ARI,-2.5,...,NaN,DOME,Cardinals,Saints,Cardinals,Cardinals,-2.5,H,28.5,19
2,09/13/2015,2015,1,False,Buffalo Bills,27,14,Indianapolis Colts,IND,-1.0,...,NaN,NaN,Bills,Colts,Colts,Bills,1.0,H,28.0,14
3,09/13/2015,2015,1,False,Chicago Bears,23,31,Green Bay Packers,GB,-6.5,...,NaN,NaN,Bears,Packers,Packers,Bears,6.5,H,29.5,31
4,09/13/2015,2015,1,False,Dallas Cowboys,27,26,New York Giants,DAL,-7.0,...,NaN,DOME,Cowboys,Giants,Cowboys,Cowboys,-7.0,H,20.0,26


In [141]:
df2['result']=0
df2.loc[df2['target_adj_score']>df2['opposition_score'], 'result']='W'
df2.loc[df2['target_adj_score']<=df2['opposition_score'], 'result']='L'

In [143]:
df2['date_formatted']=pd.to_datetime(df2['schedule_date'])

In [144]:
df2.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,team_home_fix,team_away_fix,team_favorite_id_fix,target,target_spread,home_away,target_adj_score,opposition_score,result,date_formatted
0,09/10/2015,2015,1,False,New England Patriots,28,21,Pittsburgh Steelers,NE,-7.0,...,Patriots,Steelers,Patriots,Patriots,-7.0,H,21.0,21,L,2015-09-10
1,09/13/2015,2015,1,False,Arizona Cardinals,31,19,New Orleans Saints,ARI,-2.5,...,Cardinals,Saints,Cardinals,Cardinals,-2.5,H,28.5,19,W,2015-09-13
2,09/13/2015,2015,1,False,Buffalo Bills,27,14,Indianapolis Colts,IND,-1.0,...,Bills,Colts,Colts,Bills,1.0,H,28.0,14,W,2015-09-13
3,09/13/2015,2015,1,False,Chicago Bears,23,31,Green Bay Packers,GB,-6.5,...,Bears,Packers,Packers,Bears,6.5,H,29.5,31,L,2015-09-13
4,09/13/2015,2015,1,False,Dallas Cowboys,27,26,New York Giants,DAL,-7.0,...,Cowboys,Giants,Cowboys,Cowboys,-7.0,H,20.0,26,L,2015-09-13


# Begin adding other statsitics from follow up CSVs

## Adding offensive stats:

In [145]:
df_add = pd.read_csv(folder+'data_sources/team_year_end_stats.csv')

In [146]:
df_add.head()

,year (start),rank,team,games_played,points_scored,yards,offensive_plays,yards_per_offensive_play,turnovers_lost,fumbles_lost,...,rushing_yards,rushing_touchdowns,rushing_yards_per_attempt,first_downs_by_rushing,penalties_committed,penalty_yards,first_downs_from_penalty,percent_of_drives_with_offensive_score,percent_drives_ending_in_turnover,extra_points
0,2020,1,Seattle Seahawks,5,169,1979,302,6.6,4,1,...,577,4,4.9,34,23,169,7,44.6,7.1,72.74
1,2020,2,Dallas Cowboys,5,163,2440,373,6.5,11,7,...,533,9,4.2,38,33,274,11,45.8,18.6,62.08
2,2020,3,Cleveland Browns,5,156,1933,333,5.8,6,2,...,942,8,5.5,49,29,256,8,46.3,11.1,58.89
3,2020,4,New Orleans Saints,5,153,1892,325,5.8,4,1,...,575,8,4.1,37,33,458,7,48.1,7.4,56.87
4,2020,5,Green Bay Packers,4,152,1782,261,6.8,0,0,...,603,4,5.1,30,19,163,12,64.1,0.0,93.24


In [149]:
df_add['name_adj'] = df_add['team'].str.split(' ')

In [152]:
df_add['name_adj'] = df_add.name_adj.str[-1]

In [153]:
df_add.head()

,year (start),rank,team,games_played,points_scored,yards,offensive_plays,yards_per_offensive_play,turnovers_lost,fumbles_lost,...,rushing_touchdowns,rushing_yards_per_attempt,first_downs_by_rushing,penalties_committed,penalty_yards,first_downs_from_penalty,percent_of_drives_with_offensive_score,percent_drives_ending_in_turnover,extra_points,name_adj
0,2020,1,Seattle Seahawks,5,169,1979,302,6.6,4,1,...,4,4.9,34,23,169,7,44.6,7.1,72.74,Seahawks
1,2020,2,Dallas Cowboys,5,163,2440,373,6.5,11,7,...,9,4.2,38,33,274,11,45.8,18.6,62.08,Cowboys
2,2020,3,Cleveland Browns,5,156,1933,333,5.8,6,2,...,8,5.5,49,29,256,8,46.3,11.1,58.89,Browns
3,2020,4,New Orleans Saints,5,153,1892,325,5.8,4,1,...,8,4.1,37,33,458,7,48.1,7.4,56.87,Saints
4,2020,5,Green Bay Packers,4,152,1782,261,6.8,0,0,...,4,5.1,30,19,163,12,64.1,0.0,93.24,Packers


In [154]:
df_add.columns

Index(['year (start)', 'rank', 'team', 'games_played', 'points_scored',
       'yards', 'offensive_plays', 'yards_per_offensive_play',
       'turnovers_lost', 'fumbles_lost', 'first_down', 'pass_completed',
       'passes_attempted', 'yards_gained_passing', 'passing_touchdowns',
       'interceptions_thrown', 'net_gain_from_passess_and_sacks',
       'first_down_passing', 'rusing_attempts', 'rushing_yards',
       'rushing_touchdowns', 'rushing_yards_per_attempt',
       'first_downs_by_rushing', 'penalties_committed', 'penalty_yards',
       'first_downs_from_penalty', 'percent_of_drives_with_offensive_score',
       'percent_drives_ending_in_turnover', 'extra_points', 'name_adj'],
      dtype='object')

In [155]:
df_add.columns = ['year', 'offense_rank', 'team', 'games_played', 'points_scored',
       'yards', 'offensive_plays', 'yards_per_offensive_play',
       'offense_turnovers_lost', 'offense_fumbles_lost', 'offense_first_down', 'offense_pass_completed',
       'offense_passes_attempted', 'offense_yards_gained_passing', 'offense_passing_touchdowns',
       'offense_interceptions_thrown', 'offense_ net_gain_from_passess_and_sacks',
       'offense_first_down_passing', 'offense_rusing_attempts', 'offense_rushing_yards',
       'offense_rushing_touchdowns', 'offense_rushing_yards_per_attempt',
       'offense_first_downs_by_rushing', 'offense_penalties_committed', 'offense_penalty_yards',
       'offense_first_downs_from_penalty', 'offense_percent_of_drives_with_offensive_score',
       'offense_percent_drives_ending_in_turnover', 'offense_extra_points', 'name_adj']

In [157]:
df3 = df2.merge(df_add, how = 'inner', left_on= ['schedule_season', 'target'], right_on=['year', 'name_adj'])

array(['Patriots', 'Cardinals', 'Bills', 'Bears', 'Cowboys', 'Broncos',
       'Texans', 'Jaguars', 'Jets', 'Raiders', 'Chargers', 'Rams',
       'Buccaneers', 'Redskins', 'Falcons', '49ers', 'Chiefs', 'Panthers',
       'Bengals', 'Browns', 'Packers', 'Vikings', 'Saints', 'Giants',
       'Eagles', 'Steelers', 'Colts', 'Ravens', 'Lions', 'Dolphins',
       'Seahawks', 'Titans'], dtype=object)

## Offense Passing Stats

In [164]:
df_offense_passing = pd.read_csv(folder+'data_sources/offense_passing.csv')

In [166]:
df_offense_passing.head()

,Year,Rank,Team,games_played,passes_completed,passes_attempted,percent_pass_completed,yards_gained_passing,passing_touchdowns,passing_touchdown_percentage,...,yards_gained_per_game_played,quarterback_rating,times_sacked,yards_lost_due_to_sacks,NY/A,net_yards_gained_per_pass_attempt,percentage_of_times_sacked_while_passing,comebacks_led_by_QB,game_winning_drives_by_qb,expected_points_contributed_by_passing_offense
0,2020,1,Dallas Cowboys,5,161,236,68.2,1907,10,4.2,...,381.4,100.9,11,71,7.7,7.8,4.5,2.0,2.0,47.68
1,2020,2,Buffalo Bills,5,134,194,69.1,1540,14,7.2,...,308.0,111.9,10,71,7.5,8.3,4.9,2.0,2.0,84.84
2,2020,3,Kansas City Chiefs,5,123,193,63.7,1439,13,6.7,...,287.8,107.3,6,35,7.2,8.3,3.0,1.0,1.0,77.65
3,2020,4,Carolina Panthers,5,130,178,73.0,1409,6,3.4,...,281.8,101.3,8,51,7.6,7.5,4.3,NaN,NaN,51.71
4,2020,5,Seattle Seahawks,5,123,169,72.8,1402,19,11.2,...,280.4,129.8,15,100,7.6,9.0,8.2,2.0,2.0,72.50


In [167]:
df_offense_passing['name_adj'] = df_offense_passing.Team.str.split(' ')

In [168]:
df_offense_passing['name_adj']=df_offense_passing.name_adj.str[-1]

In [170]:
df_offense_passing.columns

Index(['Year', 'Rank', 'Team', 'games_played', 'passes_completed',
       'passes_attempted', 'percent_pass_completed', 'yards_gained_passing',
       'passing_touchdowns', 'passing_touchdown_percentage',
       'interceptions_thrown', 'percent_passes_intercepted',
       'longest_completed_pass', 'yards_gained_per_pass_attempt',
       'adjusted_yards_gained_per_pass_attempt',
       'yards_gained_per_pass_completion', 'yards_gained_per_game_played',
       'quarterback_rating', 'times_sacked', 'yards_lost_due_to_sacks', 'NY/A',
       'net_yards_gained_per_pass_attempt',
       'percentage_of_times_sacked_while_passing', 'comebacks_led_by_QB',
       'game_winning_drives_by_qb',
       'expected_points_contributed_by_passing_offense', 'name_adj'],
      dtype='object')

In [172]:
df_offense_passing.columns = ['Year', 'target_passing_rank', 'Team', 'target_games_played', 'target_offense_passes_completed',
       'target_offese_passes_attempted', 'target_offense_percent_pass_completed', 'target_yards_gained_passing',
       'target_offense_passing_touchdowns', 'target_offense_passing_touchdown_percentage',
       'target_offense_interceptions_thrown', 'target_offense_percent_passes_intercepted',
       'target_offense_longest_completed_pass', 'target_offense_yards_gained_per_pass_attempt',
       'target_offense_adjusted_yards_gained_per_pass_attempt',
       'target_offense_yards_gained_per_pass_completion', 'target_offense_yards_gained_per_game_played',
       'target_offense_quarterback_rating', 'target_offense_times_sacked', 'target_offense_yards_lost_due_to_sacks', 'target_offense_NY/A',
       'target_offense_net_yards_gained_per_pass_attempt',
       'target_offense_percentage_of_times_sacked_while_passing', 'target_offense_comebacks_led_by_QB',
       'target_offense_game_winning_drives_by_qb',
       'target_offense_expected_points_contributed_by_passing_offense', 'target_name_adj']

In [177]:
df4 = df3.merge(df_offense_passing, how ='inner', left_on = ['schedule_season', 'target'], right_on = ['Year', 'target_name_adj'])

In [174]:
df3.columns

Index(['schedule_date', 'schedule_season', 'schedule_week', 'schedule_playoff',
       'team_home', 'score_home', 'score_away', 'team_away',
       'team_favorite_id', 'spread_favorite', 'over_under_line', 'stadium',
       'stadium_neutral', 'weather_temperature', 'weather_wind_mph',
       'weather_humidity', 'weather_detail', 'team_home_fix', 'team_away_fix',
       'team_favorite_id_fix', 'target', 'target_spread', 'home_away',
       'target_adj_score', 'opposition_score', 'result', 'date_formatted',
       'year', 'offense_rank', 'team', 'games_played', 'points_scored',
       'yards', 'offensive_plays', 'yards_per_offensive_play',
       'offense_turnovers_lost', 'offense_fumbles_lost', 'offense_first_down',
       'offense_pass_completed', 'offense_passes_attempted',
       'offense_yards_gained_passing', 'offense_passing_touchdowns',
       'offense_interceptions_thrown',
       'offense_ net_gain_from_passess_and_sacks',
       'offense_first_down_passing', 'offense_rusing_at

(2670, 84)

## Offensive Rushing

In [188]:
df_offensive_rushing = pd.read_csv(folder+'data_sources/offense_rushing.csv')

In [190]:
df_offensive_rushing.head()

,Year,Rank,team,games_played,rushing_attempts,rushing_yards_gained,rushing_touchdowns,longest_rushing_attempt,Y/A,Y/G,Fmb,EXP
0,2020,1,Cleveland Browns,5,172,942,8,50,5.5,188.4,5.0,22.40
1,2020,2,Baltimore Ravens,5,144,804,5,50,5.6,160.8,5.0,21.34
2,2020,3,Minnesota Vikings,5,154,803,8,39,5.2,160.6,4.0,18.24
3,2020,4,New England Patriots,4,140,719,7,48,5.1,179.8,3.0,31.06
4,2020,5,Arizona Cardinals,5,148,705,8,48,4.8,141.0,5.0,23.33


In [192]:
df_offensive_rushing['name_adj']=df_offensive_rushing.team.str.split(' ')

In [193]:
df_offensive_rushing['name_adj']=df_offensive_rushing.name_adj.str[-1]

In [194]:
df_offensive_rushing.columns

Index(['Year', 'Rank', 'team', 'games_played', 'rushing_attempts',
       'rushing_yards_gained', 'rushing_touchdowns', 'longest_rushing_attempt',
       'Y/A', 'Y/G', 'Fmb', 'EXP', 'name_adj'],
      dtype='object')

In [195]:
df_offensive_rushing.columns=['Year', 'target_offenseive_rushing_Rank', 'team', 'target_games_played', 'target_offensive_rushing_attempts',
       'target_offensive_rushing_yards_gained', 'target_offensive_rushing_touchdowns', 'target_offensive_longest_rushing_attempt',
       'target_offensive_rushing_Y/A', 'target_offensive_rushing_Y/G', 'target_offensive_rushing_Fmb', 'target_offensive_rushing_EXP', 'name_adj']

In [198]:
df5 = df4.merge(df_offensive_rushing, how='inner', left_on = ['schedule_season', 'target'], right_on = ['Year', 'name_adj'])

In [200]:
df5.shape

(2670, 97)

##  fixing offense columns to reflect that they represent statistics for the target team

In [201]:
df5.columns

Index(['schedule_date', 'schedule_season', 'schedule_week', 'schedule_playoff',
       'team_home', 'score_home', 'score_away', 'team_away',
       'team_favorite_id', 'spread_favorite', 'over_under_line', 'stadium',
       'stadium_neutral', 'weather_temperature', 'weather_wind_mph',
       'weather_humidity', 'weather_detail', 'team_home_fix', 'team_away_fix',
       'team_favorite_id_fix', 'target', 'target_spread', 'home_away',
       'target_adj_score', 'opposition_score', 'result', 'date_formatted',
       'year', 'offense_rank', 'team_x', 'games_played', 'points_scored',
       'yards', 'offensive_plays', 'yards_per_offensive_play',
       'offense_turnovers_lost', 'offense_fumbles_lost', 'offense_first_down',
       'offense_pass_completed', 'offense_passes_attempted',
       'offense_yards_gained_passing', 'offense_passing_touchdowns',
       'offense_interceptions_thrown',
       'offense_ net_gain_from_passess_and_sacks',
       'offense_first_down_passing', 'offense_rusing_

In [202]:
df5.columns = ['schedule_date', 'schedule_season', 'schedule_week', 'schedule_playoff',
       'team_home', 'score_home', 'score_away', 'team_away',
       'team_favorite_id', 'spread_favorite', 'over_under_line', 'stadium',
       'stadium_neutral', 'weather_temperature', 'weather_wind_mph',
       'weather_humidity', 'weather_detail', 'team_home_fix', 'team_away_fix',
       'team_favorite_id_fix', 'target', 'target_spread', 'home_away',
       'target_adj_score', 'opposition_score', 'result', 'date_formatted',
       'year', 'offense_rank', 'team_x', 'target_games_played', 'points_scored',
       'yards', 'target_offensive_plays', 'target_yards_per_offensive_play',
       'target_offense_turnovers_lost', 'target_offense_fumbles_lost', 'target_offense_first_down',
       'target_offense_pass_completed', 'target_offense_passes_attempted',
       'target_offense_yards_gained_passing', 'target_offense_passing_touchdowns',
       'target_offense_interceptions_thrown',
       'target_offense_ net_gain_from_passess_and_sacks',
       'target_offense_first_down_passing', 'target_offense_rusing_attempts',
       'target_offense_rushing_yards', 'target_offense_rushing_touchdowns',
       'target_offense_rushing_yards_per_attempt', 'target_offense_first_downs_by_rushing',
       'target_offense_penalties_committed', 'target_offense_penalty_yards',
       'target_offense_first_downs_from_penalty',
       'target_offense_percent_of_drives_with_offensive_score',
       'offense_percent_drives_ending_in_turnover', 'offense_extra_points',
       'name_adj_x', 'Year_x', 'target_passing_rank', 'Team',
       'target_games_played_x', 'target_offense_passes_completed',
       'target_offese_passes_attempted',
       'target_offense_percent_pass_completed', 'target_yards_gained_passing',
       'target_offense_passing_touchdowns',
       'target_offense_passing_touchdown_percentage',
       'target_offense_interceptions_thrown',
       'target_offense_percent_passes_intercepted',
       'target_offense_longest_completed_pass',
       'target_offense_yards_gained_per_pass_attempt',
       'target_offense_adjusted_yards_gained_per_pass_attempt',
       'target_offense_yards_gained_per_pass_completion',
       'target_offense_yards_gained_per_game_played',
       'target_offense_quarterback_rating', 'target_offense_times_sacked',
       'target_offense_yards_lost_due_to_sacks', 'target_offense_NY/A',
       'target_offense_net_yards_gained_per_pass_attempt',
       'target_offense_percentage_of_times_sacked_while_passing',
       'target_offense_comebacks_led_by_QB',
       'target_offense_game_winning_drives_by_qb',
       'target_offense_expected_points_contributed_by_passing_offense',
       'target_name_adj', 'Year_y', 'target_offenseive_rushing_Rank', 'team_y',
       'target_games_played_y', 'target_offensive_rushing_attempts',
       'target_offensive_rushing_yards_gained',
       'target_offensive_rushing_touchdowns',
       'target_offensive_longest_rushing_attempt',
       'target_offensive_rushing_Y/A', 'target_offensive_rushing_Y/G',
       'target_offensive_rushing_Fmb', 'target_offensive_rushing_EXP',
       'name_adj_y']

## Target defense stats:

In [203]:
df_target_defense = pd.read_csv(folder+'data_sources/team_defense.csv')

In [204]:
df_target_defense.head()

,Year,Rank,Team,games_played,points_scored_by_team,yards_per_game,offensive_plays,yards_per_offensive_plays,takeaways,fumbles_lost_by_team,...,rushing_yards_gained,rushing_tds,rushing_yards_per_attempt,first_downs_by_rushing,penalties_commited_by_team,penalties_commited_by_team_yards,first_downs_by_penalty,percent_of_drives_ending_in_offensive_score,percentage_of_drives_ending_in_offensive_turnover,expected_points_contributed_by_all_defense
0,2020,1,Baltimore Ravens,5,76,1675,333,5.0,10,7,...,460,3,3.7,22,25,200,8,25.9,18.5,13.49
1,2020,2,Pittsburgh Steelers,4,87,1206,240,5.0,7,1,...,256,3,3.3,12,21,188,10,31.8,15.9,-6.80
2,2020,3,Indianapolis Colts,5,88,1330,285,4.7,9,0,...,432,1,3.6,23,29,276,9,30.0,18.0,13.69
3,2020,4,Tennessee Titans,4,90,1637,264,6.2,9,3,...,593,3,5.5,32,26,192,3,36.4,18.2,-33.47
4,2020,5,Los Angeles Rams,5,90,1521,314,4.8,6,2,...,532,5,4.7,37,24,195,7,32.7,11.5,1.73


In [206]:
df_target_defense['name_adj'] = df_target_defense.Team.str.split(' ')
df_target_defense['name_adj'] = df_target_defense.name_adj.str[-1]

In [207]:
df_target_defense.columns

Index(['Year', 'Rank', 'Team', 'games_played', 'points_scored_by_team',
       'yards_per_game', 'offensive_plays', 'yards_per_offensive_plays',
       'takeaways', 'fumbles_lost_by_team', '1stD', 'passes_completed',
       'passes_attempted', 'yards_gained_by_passing', 'passing_touchdown',
       'interceptions_thrown', 'net_yards_gained_per_pass_attempt',
       'first_downs_by_passing', 'rushing_attempts', 'rushing_yards_gained',
       'rushing_tds', 'rushing_yards_per_attempt', 'first_downs_by_rushing',
       'penalties_commited_by_team', 'penalties_commited_by_team_yards',
       'first_downs_by_penalty', 'percent_of_drives_ending_in_offensive_score',
       'percentage_of_drives_ending_in_offensive_turnover',
       'expected_points_contributed_by_all_defense', 'name_adj'],
      dtype='object')

In [208]:
df_target_defense.columns = ['Year', 'target_defense_rank', 'Team', 'games_played', 'target_defense_points_scored_by_team',
       'target_defense)_yards_per_game', 'target_defense_offensive_plays', 'target_defense_yards_per_offensive_plays',
       'target_defense_takeaways', 'target_defense_fumbles_lost_by_team', 'target_defense_1stD', 'target_defense_passes_completed',
       'target_defense_passes_attempted', 'target_defense_yards_gained_by_passing', 'target_defense_passing_touchdown',
       'target_defense_interceptions_thrown', 'target_defense_net_yards_gained_per_pass_attempt',
       'target_defense_first_downs_by_passing', 'target_defense_rushing_attempts', 'target_defense_rushing_yards_gained',
       'target_defense_rushing_tds', 'target_defense_rushing_yards_per_attempt', 'target_defense_first_downs_by_rushing',
       'target_defense_penalties_commited_by_team', 'target_defense_penalties_commited_by_team_yards',
       'target_defense_first_downs_by_penalty', 'target_defense_percent_of_drives_ending_in_offensive_score',
       'target_defense_percentage_of_drives_ending_in_offensive_turnover',
       'target_defense_expected_points_contributed_by_all_defense', 'target_defense_name_adj']

In [211]:
df6 = df5.merge(df_target_defense, how = 'inner', left_on = ['schedule_season', 'target'], right_on = ['Year','target_defense_name_adj'])

In [213]:
df6.shape

(2670, 127)

# Opponent info

## Offense

In [215]:
df6['opponent'] = df6.team_away.str.split(' ')
df6['opponent'] = df6.opponent.str[-1]

In [216]:
df_add_op = df_add.copy()

In [217]:
df_add_op.columns = ['year', 'opponent_offense_rank', 'opponent_team', 'opponent_games_played', 'opponent_points_scored',
       'opponent_yards', 'opponent_offensive_plays', 'opponent_yards_per_offensive_play',
       'opponent_offense_turnovers_lost', 'opponent_offense_fumbles_lost', 'opponent_offense_first_down', 'opponent_offense_pass_completed',
       'opponent_offense_passes_attempted', 'opponent_offense_yards_gained_passing', 'opponent_offense_passing_touchdowns',
       'opponent_offense_interceptions_thrown', 'opponent_offense_ net_gain_from_passess_and_sacks',
       'opponent_offense_first_down_passing', 'opponent_offense_rusing_attempts', 'opponent_offense_rushing_yards',
       'opponent_offense_rushing_touchdowns', 'opponent_offense_rushing_yards_per_attempt',
       'opponent_offense_first_downs_by_rushing', 'opponent_offense_penalties_committed', 'opponent_offense_penalty_yards',
       'opponent_offense_first_downs_from_penalty', 'opponent_offense_percent_of_drives_with_offensive_score',
       'opponent_offense_percent_drives_ending_in_turnover', 'opponent_offense_extra_points', 'opponent_name_adj']

In [218]:
df7 = df6.merge(df_add_op, how='inner', left_on = ['schedule_season', 'opponent'], right_on = ['year', 'opponent_name_adj'])

In [219]:
df7.shape

(2670, 158)

## Defense

In [220]:
df_opponent_defense = df_target_defense.copy()

In [221]:
df_opponent_defense.columns = ['Year', 'opponent_defense_Rank', 'opponent_defense_Team', 'opponent_defense_games_played', 'opponent_defense_points_scored_by_team',
       'opponent_defense_yards_per_game', 'opponent_defense_offensive_plays', 'opponent_defense_yards_per_offensive_plays',
       'opponent_defense_takeaways', 'opponent_defense_fumbles_lost_by_team', 'opponent_defense_1stD', 'opponent_defense_passes_completed',
       'opponent_defense_passes_attempted', 'opponent_defense_yards_gained_by_passing', 'opponent_defense_passing_touchdown',
       'opponent_defense_interceptions_thrown', 'opponent_defense_net_yards_gained_per_pass_attempt',
       'opponent_defense_first_downs_by_passing', 'opponent_defense_rushing_attempts', 'opponent_defense_rushing_yards_gained',
       'opponent_defense_rushing_tds', 'opponent_defense_rushing_yards_per_attempt', 'opponent_defense_first_downs_by_rushing',
       'opponent_defense_penalties_commited_by_team', 'opponent_defense_penalties_commited_by_team_yards',
       'opponent_defense_first_downs_by_penalty', 'opponent_defense_percent_of_drives_ending_in_offensive_score',
       'opponent_defense_percentage_of_drives_ending_in_offensive_turnover',
       'opponent_defense_expected_points_contributed_by_all_defense', 'opponent_defense_name_adj']

In [223]:
df8 = df7.merge(df_opponent_defense, how='inner', left_on =['schedule_season', 'opponent'], right_on = ['Year', 'opponent_defense_name_adj'])

In [224]:
df8.shape

(2670, 188)

# Saving data for MPV:

In [225]:
df8.to_csv(folder+'data_cleaning/mvp_data.csv')

# Creating usable databse for analysis:

In [229]:
print(df8.columns)

Index(['schedule_date', 'schedule_season', 'schedule_week', 'schedule_playoff',
       'team_home', 'score_home', 'score_away', 'team_away',
       'team_favorite_id', 'spread_favorite',
       ...
       'opponent_defense_rushing_tds',
       'opponent_defense_rushing_yards_per_attempt',
       'opponent_defense_first_downs_by_rushing',
       'opponent_defense_penalties_commited_by_team',
       'opponent_defense_penalties_commited_by_team_yards',
       'opponent_defense_first_downs_by_penalty',
       'opponent_defense_percent_of_drives_ending_in_offensive_score',
       'opponent_defense_percentage_of_drives_ending_in_offensive_turnover',
       'opponent_defense_expected_points_contributed_by_all_defense',
       'opponent_defense_name_adj'],
      dtype='object', length=188)


In [233]:
df8['home?']=0
df8['away?']=0
df8.loc[df8['home_away']=='H', 'home?']=1
df8.loc[df8['home_away']=='A', 'away?']=1

In [230]:
df8.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_home_fix,team_away_fix,team_favorite_id_fix,target,target_spread,home_away,target_adj_score,opposition_score,result,date_formatted,year_x,offense_rank,team_x,target_games_played,points_scored,yards,target_offensive_plays,target_yards_per_offensive_play,target_offense_turnovers_lost,target_offense_fumbles_lost,target_offense_first_down,target_offense_pass_completed,target_offense_passes_attempted,target_offense_yards_gained_passing,target_offense_passing_touchdowns,target_offense_interceptions_thrown,target_offense_ net_gain_from_passess_and_sacks,target_offense_first_down_passing,target_offense_rusing_attempts,target_offense_rushing_yards,target_offense_rushing_touchdowns,target_offense_rushing_yards_per_attempt,target_offense_first_downs_by_rushing,target_offense_penalties_committed,target_offense_penalty_yards,target_offense_first_downs_from_penalty,target_offense_percent_of_drives_with_offensive_score,offense_percent_drives_ending_in_turnover,offense_extra_points,name_adj_x,Year_x,target_passing_rank,Team_x,target_games_played_x,target_offense_passes_completed,target_offese_passes_attempted,target_offense_percent_pass_completed,target_yards_gained_passing,target_offense_passing_touchdowns,target_offense_passing_touchdown_percentage,target_offense_interceptions_thrown,target_offense_percent_passes_intercepted,target_offense_longest_completed_pass,target_offense_yards_gained_per_pass_attempt,target_offense_adjusted_yards_gained_per_pass_attempt,target_offense_yards_gained_per_pass_completion,target_offense_yards_gained_per_game_played,target_offense_quarterback_rating,target_offense_times_sacked,target_offense_yards_lost_due_to_sacks,target_offense_NY/A,target_offense_net_yards_gained_per_pass_attempt,target_offense_percentage_of_times_sacked_while_passing,target_offense_comebacks_led_by_QB,target_offense_game_winning_drives_by_qb,target_offense_expected_points_contributed_by_passing_offense,target_name_adj,Year_y,target_offenseive_rushing_Rank,team_y,target_games_played_y,target_offensive_rushing_attempts,target_offensive_rushing_yards_gained,target_offensive_rushing_touchdowns,target_offensive_longest_rushing_attempt,target_offensive_rushing_Y/A,target_offensive_rushing_Y/G,target_offensive_rushing_Fmb,target_offensive_rushing_EXP,name_adj_y,Year_x,target_defense_rank,Team_y,games_played,target_defense_points_scored_by_team,target_defense)_yards_per_game,target_defense_offensive_plays,target_defense_yards_per_offensive_plays,target_defense_takeaways,target_defense_fumbles_lost_by_team,target_defense_1stD,target_defense_passes_completed,target_defense_passes_attempted,target_defense_yards_gained_by_passing,target_defense_passing_touchdown,target_defense_interceptions_thrown,target_defense_net_yards_gained_per_pass_attempt,target_defense_first_downs_by_passing,target_defense_rushing_attempts,target_defense_rushing_yards_gained,target_defense_rushing_tds,target_defense_rushing_yards_per_attempt,target_defense_first_downs_by_rushing,target_defense_penalties_commited_by_team,target_defense_penalties_commited_by_team_yards,target_defense_first_downs_by_penalty,target_defense_percent_of_drives_ending_in_offensive_score,target_defense_percentage_of_drives_ending_in_offensive_turnover,target_defense_expected_points_contributed_by_all_defense,target_defense_name_adj,opponent,year_y,opponent_offense_rank,opponent_team,opponent_games_played,opponent_points_scored,opponent_yards,opponent_offensive_plays,opponent_yards_per_offensive_play,opponent_offense_turnovers_lost,opponent_offense_fumbles_lost,opponent_offense_first_down,opponent_offense_pass_completed,opponent_offense_passes_attempted,opponent_offense_yards_gained_passing,opponent_offense_passing_touchdowns,opponent_offense_interceptio

In [247]:
df_mvp = df8['over_under_line'].reset_index().drop('index', axis=1)

In [248]:
df_mvp.head()

,over_under_line
0,51
1,41
2,45.5
3,48.5
4,41.5


In [249]:
df_mvp['target_spread'] = df8['target_spread']

In [356]:
df_mvp['beat_spread'] = df8['result']

In [357]:
df_mvp.loc[df_mvp['beat_spread']=='W', 'beat_spread']=1
df_mvp.loc[df_mvp['beat_spread']!=1, 'beat_spread']=0

In [250]:
df_mvp['home_away'] = df8['home_away']

In [251]:
df_mvp['target_offense_rank'] = df8['offense_rank']

In [252]:
df_mvp['target_points_scored_per_game'] = df8['points_scored']/16

In [253]:
df_mvp['target_yards_per_game'] = df8['yards']/16

In [254]:
df_mvp['target_offensive_plays_per_game']=df8['target_offensive_plays']/16

In [256]:
df_mvp['target_yards_per_play_offense'] = df8['target_yards_per_offensive_play']

In [257]:
df_mvp['target_turnovers_per_game'] =df8['target_offense_turnovers_lost'] /16

In [258]:
df_mvp['target_offense_fumbles_lost_per_game']=df8['target_offense_fumbles_lost']/16

In [259]:
df_mvp['target_offense_first_downs_completed_per_game'] = df8['target_offense_first_down']/16

In [260]:
df_mvp['target_offense_pass_completion_percentage'] = df8['target_offense_pass_completed']/df8['target_offense_passes_attempted']

In [261]:
df_mvp['target_offense_passing_to_all_types_percentage'] = df8['target_offense_passes_attempted']/df8['target_offensive_plays']

In [262]:
df_mvp['target_offense_yards_per_passing_play'] = df8['target_offense_yards_gained_passing']/df8['target_offense_passes_attempted']

In [268]:
df_mvp['target_offense_first_downs_by_passing_percentage'] = df8['target_offense_first_down_passing']/df8['target_offense_first_down']

In [267]:
df_mvp['target_offense_quarterback_rating']= df8['target_offense_quarterback_rating']

In [269]:
df_mvp['target_offense_rushing_per_total_plays'] = df8['target_offensive_rushing_attempts']/df8['target_offense_passes_attempted']

In [271]:
#df_mvp['target_offense_interceptions_thrown_per_game'] = df8['target_offense_interceptions_thrown']/16

#need to look into why interceptions not working

ValueError: Wrong number of items passed 2, placement implies 1

In [272]:
df_mvp['target_offense_passing_rank']= df8['target_passing_rank']

In [273]:
df_mvp['target_offense_rushing_rank']=df8['target_offenseive_rushing_Rank']

In [320]:
df_mvp['target_offense_rushing_yards_per_attempt'] = df8['target_offensive_rushing_yards_gained']/df8['target_offense_rusing_attempts']

In [275]:
df_mvp['target_offense_percent_of_plays_rushing'] = df8['target_offensive_rushing_attempts']/df8['target_offensive_plays']

In [276]:
df_mvp['target_defense_rank']=df8['target_defense_rank']

In [278]:
df_mvp['target_defense_scored_against_per_game'] = df8['target_defense_points_scored_by_team']/16

In [279]:
df_mvp['target_defense_yards_against_per_game'] = df8['target_defense)_yards_per_game']/16

In [280]:
df_mvp['target_defensive_plays_per_game'] = df8['target_defense_offensive_plays']/16

In [282]:
df_mvp['target_first_downs_against_per_game'] = df8['target_defense_1stD']/16

In [283]:
df_mvp['target_defense_passes_completed_percentage'] = df8['target_defense_passes_completed']/df8['target_defense_passes_attempted']

In [284]:
df_mvp['target_defense_interceptions'] = df8['target_defense_interceptions_thrown']

In [285]:
df_mvp['target_defense_percent_of_drives_ending_in_offensive_score'] = df8['target_defense_percent_of_drives_ending_in_offensive_score']

In [286]:
df_mvp['opponent_offense_rank'] = df8['opponent_offense_rank']

In [287]:
df_mvp['opponent_points_scored_per_rank'] = df8['opponent_points_scored']/16

In [288]:
df_mvp['opponent_offensive_yards_per_game'] = df8['opponent_yards']/16

In [289]:
df_mvp['opponent_yards_per_offensive_play']=df8['opponent_yards_per_offensive_play']

In [291]:
df_mvp['opponent_offense_fumbles_lost_per_game'] = df8['opponent_offense_fumbles_lost']/16

In [292]:
df_mvp['opponent_offense_first_down_per_game'] = df8['opponent_offense_first_down']/16

In [293]:
df_mvp['opponent_offense_pass_completion_percentage'] = df8['opponent_offense_pass_completed']/df8['opponent_offense_passes_attempted']

In [295]:
df_mvp['opponent_offense_yards_gained_passing_per_passing_attempt'] = df8['opponent_offense_yards_gained_passing']/df8['opponent_offense_passes_attempted']

In [296]:
df_mvp['opponent_offense_interceptions_thrown_per_game'] = df8['opponent_offense_interceptions_thrown']/16

In [297]:
df_mvp['opponent_offense_rushing_yards_per_attempt'] = df8['opponent_offense_rushing_yards_per_attempt']

In [298]:
df_mvp['opponent_offense_rushing_attempt_total'] = df8['opponent_offense_rusing_attempts']/df8['opponent_offensive_plays']

In [299]:
df_mvp['opponent_offense_percent_of_drives_with_offensive_score'] = df8['opponent_offense_percent_of_drives_with_offensive_score']

In [300]:
df_mvp['opponent_defense_Rank'] = df8['opponent_defense_Rank']

In [301]:
df_mvp['opponent_defense_points_scored_against_per_game'] = df8['opponent_defense_points_scored_by_team']/16

In [302]:
df_mvp['opponent_defense_yards_against_per_game'] = df8['opponent_defense_yards_per_game']/16

In [303]:
df_mvp['opponent_defense_yards_per_offensive_plays'] = df8['opponent_defense_yards_per_offensive_plays']

In [304]:
df_mvp['opponent_defense_1stD_against_per_game'] = df8['opponent_defense_1stD']/16

In [305]:
df_mvp['opponent_defense_pass_completion_percentage_against'] = df8['opponent_defense_passes_completed']/df8['opponent_defense_passes_attempted']

In [306]:
df_mvp['opponent_defense_interceptions_per_game'] = df8['opponent_defense_interceptions_thrown']/16

In [308]:
df_mvp['opponent_defense_rushing_yards_gained_per_game'] = df8['opponent_defense_rushing_yards_gained']/16

In [309]:
df_mvp['opponent_defense_rushing_yards_per_attempt']=df8['opponent_defense_rushing_yards_per_attempt']

In [310]:
df_mvp['opponent_defense_percent_of_drives_ending_in_offensive_score'] = df8['opponent_defense_percent_of_drives_ending_in_offensive_score']

In [239]:
df9 = df8.drop(['schedule_date', 'schedule_season', 'schedule_week', 'schedule_playoff', 'team_home', 'score_home', 'score_away', 'team_away', 'team_favorite_id', 'spread_favorite', 'staidum','stadium_neutral', 'weather_temperature', 'weather_wind_mph', 'weather_humidity', 'weather_detail', 'team_home_fix', 'team_away_fix', 'team_favorite_id_fix','target','target_adj_score', 'opposition_score','date_formatted', 'year_x','target_games_played','name_adj_x', 'Year_x', 'Team_x','target_games_played_x','target_name_adj', 'Year_y', 'team_y', 'target_games_played_y', 'name_adj_y', 'Year_x', 'Team_y', 'games_played', 'target_defense_name_adj', 'opponent', 'year_y', 'opponent_team', 'opponent_games_played', 'opponent_name_adj', 'Year_y', 'opponent_defense_Team', 'opponent_defense_games_played', 'opponent_defense_name_adj' ], axis=1).

KeyError: "['schedule_date' 'schedule_season' 'schedule_week' 'schedule_playoff'\n 'team_home' 'score_home' 'score_away' 'team_away' 'team_favorite_id'\n 'spread_favorite' 'staidum' 'stadium_neutral' 'weather_temperature'\n 'weather_wind_mph' 'weather_humidity' 'weather_detail' 'team_home_fix'\n 'team_away_fix' 'team_favorite_id_fix' 'target' 'target_adj_score'\n 'opposition_score' 'date_formatted' 'year_x' 'target_games_played'\n 'name_adj_x' 'Year_x' 'Team_x' 'target_games_played_x' 'target_name_adj'\n 'Year_y' 'team_y' 'target_games_played_y' 'name_adj_y' 'Year_x' 'Team_y'\n 'games_played' 'target_defense_name_adj' 'opponent' 'year_y'\n 'opponent_team' 'opponent_games_played' 'opponent_name_adj' 'Year_y'\n 'opponent_defense_Team' 'opponent_defense_games_played'\n 'opponent_defense_name_adj'] not found in axis"

In [234]:
df_mvp = df8[['result', 'target_spread', 'home?', 'away?', 'offense_rank','target_games_played', 'points_scored','target_offensive_plays', 'target_yards_per_offensive_play', 'target_offense_turnovers_lost', 'target_offense_fumbles_lost', 'target_offense_first_down', 'target_offense_pass_completed', 'target offense_pass_attempted', 'target_offense_yards_gained_passing', 'target_offense_interceptions_thrown', 'target_offense_first_down_passing', 'target_offense_rushing_attempts', 'target_offense_rushing_yards', 'target_offense_rushing_touchdowns', 'target_offense_rushing_yards_per_attempt', 'target_offense_first_downs_by_rushing', 'target_offense_percent_of_drives_with_offensive_score', 'offense_percent_drives_ending_in_turnover','target_offense_percent_pass_completed', 'target_offense_passing_touchdowns', 'target_offense_percent_passes_intercepted', 'target_offense_yards_gained_per_pass_attempt', 'target_offense_quarterback_rating', 'target_offense_times_sacked', 'target_offensive_rushing_Rank', 'target_offensive_rushing_attempts', 'target_offensive_rushing_touchdowns', 'target_offensive_rushing_Y/G', 'target_defensive_rank', 'target_defense_points_scored_by_team', 'target_defensive_yards_per_offensive_plays', 'target_defense_1stD', 'target_defense_passes_completed', 'target_defense_passes_attempted', 'target_defense_passing_touchdown', 'target_defense_interceptions_thrown', 'target_defense_first_downs_by_passing', 'target_defense_rushing_attempts', 'target_defense_rushing_tds', 'target_defense_rushing_yards_per_attempt', 'target_defense_first_downs_by_rushing', 'target_defense_penalties_committed_by_team', 'target_defense_percent_of_drives_ending_in_offensive_score', 'target_defense_percentage_of_drives_ending_in_offensive_turnover', 'opponent_offense_rank', 'opponent_games_played', 'opponent_points_scored', 'opponent_yards', 'opponent_yards_per_offensive_play', 'opponent_offense_turnovers_lost', 'opponent_offense_fumbles_lost', 'opponent_offense_first_down', 'opponent_offense_pass_completed', 'opponent_offense_passes_attempted', 'opponent_offense_passing_touchdowns', 'opponent_offense_interceptions_thrown', 'opponent_offense_first_down_passing', 'opponent_offense_rusing_attempts', 'opponent_offense_rushing_yards', 'opponent_offense_rushing_touchdowns', 'opponent_offense_rushing_yards_per_attempt', 'opponent_offense_first_downs_by_rushing', 'opponent_offense_penalties_committed', 'opponent_offense_percent_of_drives_with_offensive_score', 'opponent_offense_percent_drives_ending_in_turnover', 'opponent_defense_rank', 'opponent_defense_points_scored_by_team', 'opponent_defense_yards_per_game', 'opponent_defense_yards_per_game', 'opponent_defense_offensive_plays', 'opponent_defense_yards_per_offensive_plays', 'opponent_defense_fumbles_lost_by_team', 'opponent_defense_1stD', 'opponent_defense_passes_completed', 'opponent_defense_passes_attempted', 'opponent_yards_gained_by_passing', 'opponent_defense_passing_touchdown', 'opponent_defense_interceptions_thrown', 'opponent_defense_first_downs_by_passing', 'opponent_defense_rushing_attempts', 'opponent_defense_rushing_yards_gained', 'opponent_defense_rushing_tds', 'opponent_defense_rushing_yards_per_attempt', 'opponent_defense_first_downs_by_rushing', 'opponent_defense_penalties_commited_by_team', 'opponent_defense_percento_of_drives_ending_in_offensive_score', 'opponent_defense_percentage_of_drives_ending_in_offensive_turnover']]

KeyError: "['opponent_defense_rank', 'target offense_pass_attempted', 'target_defense_penalties_committed_by_team', 'opponent_defense_percento_of_drives_ending_in_offensive_score', 'target_defensive_yards_per_offensive_plays', 'target_offense_rushing_attempts', 'opponent_yards_gained_by_passing', 'target_offensive_rushing_Rank', 'target_defensive_rank'] not in index"

# New dataframe (DF_MVP) created to show all relevant data. must whittle down to 10 instances that represent target/opponent relationships:

In [328]:
df_mvp.head()

,over_under_line,target_spread,home_away,target_offense_rank,target_points_scored_per_game,target_yards_per_game,target_offensive_plays_per_game,target_yards_per_play_offense,target_turnovers_per_game,target_offense_fumbles_lost_per_game,target_offense_first_downs_completed_per_game,target_offense_pass_completion_percentage,target_offense_passing_to_all_types_percentage,target_offense_yards_per_passing_play,target_offense_quarterback_rating,target_offense_first_downs_by_passing_percentage,target_offense_rushing_per_total_plays,target_offense_passing_rank,target_offense_rushing_rank,target_offense_rushing_yards_per_attempt,target_offense_percent_of_plays_rushing,target_defense_rank,target_defense_scored_against_per_game,target_defense_yards_against_per_game,target_defensive_plays_per_game,target_first_downs_against_per_game,target_defense_passes_completed_percentage,target_defense_interceptions,target_defense_percent_of_drives_ending_in_offensive_score,opponent_offense_rank,opponent_points_scored_per_rank,opponent_offensive_yards_per_game,opponent_yards_per_offensive_play,opponent_offense_fumbles_lost,opponent_offense_fumbles_lost_per_game,opponent_offense_first_down_per_game,opponent_offense_pass_completion_percentage,opponent_offense_yards_gained_passing_per_passing_attempt,opponent_offense_interceptions_thrown_per_game,opponent_offense_rushing_yards_per_attempt,opponent_offense_rushing_attempt_total,opponent_offense_percent_of_drives_with_offensive_score,opponent_defense_Rank,opponent_defense_points_scored_against_per_game,opponent_defense_yards_against_per_game,opponent_defense_yards_per_offensive_plays,opponent_defense_1stD_against_per_game,opponent_defense_pass_completion_percentage_against,opponent_defense_interceptions_per_game,opponent_defense_rushing_yards_gained_per_game,opponent_defense_rushing_yards_per_attempt,opponent_defense_percent_of_drives_ending_in_offensive_score,beat_spread
0,51,-7.0,H,3,29.0625,374.4375,65.6250,5.7,0.8750,0.4375,21.7500,0.642289,0.599048,7.292528,101.9,0.660920,0.608903,5,30,3.665796,0.364762,10,19.6875,339.3750,65.1250,18.9375,0.605705,12,28.9,5,26.4375,395.4375,6.3,7,0.4375,20.6875,0.662712,7.801695,1.3125,4.4,0.383778,40.5,11,19.9375,363.0625,5.5,20.4375,0.6432,1.0625,91.1875,3.8,32.3,0
1,41,-7.5,H,19,22.1875,355.5000,66.0000,5.4,1.9375,0.5000,19.6250,0.607261,0.573864,6.551155,76.3,0.640127,0.678218,14,17,4.180049,0.389205,4,18.5000,283.1250,64.5625,18.0625,0.600349,14,26.9,5,26.4375,395.4375,6.3,7,0.4375,20.6875,0.662712,7.801695,1.3125,4.4,0.383778,40.5,11,19.9375,363.0625,5.5,20.4375,0.6432,1.0625,91.1875,3.8,32.3,0
2,45.5,-4.0,H,26,20.0000,371.8125,68.7500,5.4,1.5000,0.6875,21.5000,0.662669,0.606364,6.883058,94.5,0.668605,0.589205,4,31,3.455471,0.357273,21,24.8750,361.9375,59.9375,18.6875,0.643137,11,37.8,5,26.4375,395.4375,6.3,7,0.4375,20.6875,0.662712,7.801695,1.3125,4.4,0.383778,40.5,11,19.9375,363.0625,5.5,20.4375,0.6432,1.0625,91.1875,3.8,32.3,0
3,48.5,-1.0,H,29,17.5000,297.5625,57.5000,5.2,1.3750,0.6875,15.0625,0.577167,0.514130,5.930233,74.1,0.522822,0.906977,32,7,4.559441,0.466304,13,20.6250,367.8125,68.1875,20.0000,0.668342,13,31.8,5,26.4375,395.4375,6.3,7,0.4375,20.6875,0.662712,7.801695,1.3125,4.4,0.383778,40.5,11,19.9375,363.0625,5.5,20.4375,0.6432,1.0625,91.1875,3.8,32.3,0
4,41.5,-3.5,H,9,25.3125,331.1875,59.6875,5.5,0.9375,0.5000,18.9375,0.655391,0.495288,6.881607,95.4,0.511551,0.921776,30,6,4.688073,0.456545,3,17.9375,329.3125,64.8125,19.5625,0.574959,22,27.3,5,26.4375,395.4375,6.3,7,0.4375,20.6875,0.662712,7.801695,1.3125,4.4,0.383778,40.5,11,19.9375,363.0625,5.5,20.4375,0.6432,1.0625,91.1875,3.8,32.3,0


In [315]:
df_vars = df_mvp['over_under_line'].reset_index().drop('index', axis=1).copy()

In [317]:
df_vars['offensive_defensive_rank_diff'] = df_mvp['opponent_defense_Rank'] - df_mvp['target_offense_rank']

In [318]:
df_vars['offensive_offensive_rank_dif'] = df_mvp['opponent_offense_rank'] - df_mvp['target_offense_rank']

In [319]:
df_vars['offense_defense_passing_completion'] = (df_mvp['target_offense_pass_completion_percentage']/df_mvp['opponent_defense_pass_completion_percentage_against'])-1

In [322]:
df_vars['offense_rushing_vs_defense_rushing_allowed_yards'] = df_mvp['target_offense_rushing_yards_per_attempt']/df_mvp['opponent_defense_rushing_yards_per_attempt']

In [323]:
df_vars['defense_offense_rank_dif'] = df_mvp['opponent_offense_rank']-df_mvp['target_defense_rank']

In [324]:
df_vars['QB_rating']=df_mvp['target_offense_quarterback_rating']

In [358]:
df_vars['beat_spread'] = df_mvp['beat_spread']

In [331]:
df_vars['target_spread'] = df_mvp['target_spread']

In [359]:
df_vars.head()

,over_under_line,offensive_defensive_rank_diff,offensive_offensive_rank_dif,offense_defense_passing_completion,offense_rushing_vs_defense_rushing_allowed_yards,defense_offense_rank_dif,QB_rating,beat_spread,target_spread
0,51,8,2,-0.001416,0.964683,-5,101.9,0,-7.0
1,41,-8,-14,-0.055876,1.100013,1,76.3,0,-7.5
2,45.5,-15,-21,0.030268,0.909334,-16,94.5,0,-4.0
3,48.5,-18,-24,-0.102663,1.199853,-8,74.1,0,-1.0
4,41.5,2,-4,0.018954,1.233704,2,95.4,1,-3.5


# Set up model to be trained/tested

In [334]:
from sklearn.model_selection import train_test_split

In [360]:
X = df_vars.drop('beat_spread', axis=1).copy()
y = df_vars['beat_spread'].astype(int)

X, X_test, y, y_test = train_test_split(X, y, test_size=0.33, random_state=30)

In [361]:
from sklearn.linear_model import LogisticRegressionCV

In [362]:
clf = LogisticRegressionCV(cv=5, random_state=0).fit(X, y)

In [343]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)

ValueError: Unknown label type: 'unknown'

In [346]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [365]:
lr = LogisticRegression()
lr.fit(X, y).score(X,y)

0.5788590604026845

In [350]:
y.unique(

array([0])

In [366]:
df_vars.to_csv(folder+'/data_cleaning/mvp_grid_final.csv')

# Historical Analysis

In [ ]:
df_old = 